In [1]:
from models import *
from utils import *
import torch
import torch.nn as nn

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


Load the independent student (VGG11), the teacher (VGG16), and the distilled models (LM,Hints,CRD)

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
s_i = vgg11_bn_f(False,100).to(device)
s_i.load_state_dict(torch.load('vgg11_cifar100_student.pth'))
t = vgg16_bn_f(False,100).to(device)
t.load_state_dict(torch.load('teacher_model_vgg16_bn.pth'))
s_lm = vgg11_bn_f(False,100).to(device)
s_lm.load_state_dict(torch.load('vgg11_cifar100_blm.pth'))
s_hint = vgg11_bn_f(False,100).to(device)
s_hint.load_state_dict(torch.load('vgg11_cifar100_hint_student.pth'))
s_crd = vgg11_bn_f(False,100).to(device)
s_crd.load_state_dict(torch.load('vgg11_cifar100_crd_student.pth'))

C:\Users\PCF\AppData\Local\Temp\ipykernel_33632\2524539570.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  s_i.load_state_dict(torch.load('vgg11_cifar100_student.pth'))


<All keys matched successfully>

In [3]:
print("Teacher Test Accuracy: ", eval_f(t, test_loader,nn.CrossEntropyLoss(),device)[1])
print("Independent Student Test Accuracy: ", eval_f(s_i, test_loader,nn.CrossEntropyLoss(),device)[1])
print("Basic Logit Matching Test Accuracy: ", eval_f(s_lm, test_loader,nn.CrossEntropyLoss(),device)[1])
print("Hint Test Accuracy: ", eval_f(s_hint, test_loader,nn.CrossEntropyLoss(),device)[1])
print("CRD Test Accuracy: ", eval_f(s_crd, test_loader,nn.CrossEntropyLoss(),device)[1])

Teacher Test Accuracy:  0.617
Independent Student Test Accuracy:  0.3225
Basic Logit Matching Test Accuracy:  0.47529998
Hint Test Accuracy:  0.3881
CRD Test Accuracy:  0.419


In [4]:
for param in t.parameters():
    param.requires_grad = False
for param in s_i.parameters():
    param.requires_grad = False
for param in s_lm.parameters():
    param.requires_grad = False
for param in s_hint.parameters():
    param.requires_grad = False
for param in s_crd.parameters():
    param.requires_grad = False
eval_sim_s_i = evaluate_similarities(t,s_i,test_loader)
eval_sim_s_lm = evaluate_similarities(t,s_lm,test_loader)
eval_sim_s_hint = evaluate_similarities(t,s_hint,test_loader)
eval_sim_s_crd = evaluate_similarities(t,s_crd,test_loader)

In [5]:
print("For the following metrics, the lower the value the better the similarity between the student and teacher model.")
print("KL Divergence:")
print("Independent Student: ",eval_sim_s_i[0])
print("Basic Logit Matching: ",eval_sim_s_lm[0])
print("Hint: ",eval_sim_s_hint[0])
print("CRD: ",eval_sim_s_crd[0])
print("Two Sample KS Test:")
print("Independent Student: ",eval_sim_s_i[1])
print("Basic Logit Matching: ",eval_sim_s_lm[1])
print("Hint: ",eval_sim_s_hint[1])
print("CRD: ",eval_sim_s_crd[1])
print("Wasserstein Distance:")
print("Independent Student: ",eval_sim_s_i[2])
print("Basic Logit Matching: ",eval_sim_s_lm[2])
print("Hint: ",eval_sim_s_hint[2])
print("CRD: ",eval_sim_s_crd[2])
print("For the following metrics, the higher the value the better the similarity between the student and teacher model.")
print("Spearmans Rank Correlation:")
print("Independent Student: ",eval_sim_s_i[3])
print("Basic Logit Matching: ",eval_sim_s_lm[3])
print("Hint: ",eval_sim_s_hint[3])
print("CRD: ",eval_sim_s_crd[3])

For the following metrics, the lower the value the better the similarity between the student and teacher model.
KL Divergence:
Independent Student:  0.0032531294369607406
Basic Logit Matching:  0.002604666963118799
Hint:  0.002953586064777367
CRD:  0.0028270091105157594
Two Sample KS Test:
Independent Student:  0.4900487659235669
Basic Logit Matching:  0.45020800159235647
Hint:  0.4530185111464968
CRD:  0.4549482484076433
Wasserstein Distance:
Independent Student:  0.009407327998410825
Basic Logit Matching:  0.0056603262461149715
Hint:  0.007995600135410265
CRD:  0.007594640925997358
For the following metrics, the higher the value the better the similarity between the student and teacher model.
Spearmans Rank Correlation:
Independent Student:  0.5160102829550471
Basic Logit Matching:  0.6614622055358402
Hint:  0.5321544232448726
CRD:  0.5437021301652458
